In [1]:
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
from pylab import rcParams 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates
from arch import arch_model
from scipy import stats
import statsmodels.graphics.tsaplots as sgt
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, q_stat, adfuller
from scipy.stats import probplot, moment
from arch import arch_model
from arch.univariate import ConstantMean, GARCH, Normal
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf

In [9]:
def stock_data(stock_name):
    df = yf.download(stock_name, start="2010-01-01", end="2021-04-30")
    df['Return'] = df['Close'].pct_change().mul(100)
    df = df.dropna()
    train,test = df[df.index<'2021-01-01'],df[df.index>='2021-01-01']
    return train['Return'], test['Return']

In [10]:
train, test = stock_data('AAPL')

[*********************100%***********************]  1 of 1 completed


In [99]:
def garch_model(train, num_of_predictions):
    data = pd.concat([train, test], axis=0)
    basic_gm = arch_model(data, p=1, q=1, mean='ARX',
                          vol = 'GARCH', dist = 'normal')
    start_loc = 0
    end_loc = np.where(data.index >= test.index[0])[0].min()
    forecasts = {}
    for i in range(num_of_predictions):
        sys.stdout.write('.')
        sys.stdout.flush()
        res = basic_gm.fit(first_obs=i, last_obs=i + end_loc, disp='off', update_freq = 5)
        temp = res.forecast(horizon=1).mean
        
        fcast = temp.iloc[i + end_loc - 1]
        forecasts[fcast.name] = fcast
    rolling = pd.DataFrame(forecasts).T
    return rolling

In [100]:
predict = garch_model(train, len(test))['h.1'].values

.................................................................................

In [103]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

def evaluation(observation, forecast): 

    mae = mean_absolute_error(observation, forecast)
    print(f'Mean Absolute Error (MAE): {round(mae,4)}')

    mse = mean_squared_error(observation, forecast)
    print(f'Mean Squared Error (MSE): {round(mse,4)}')
    
    mape = mean_absolute_percentage_error(observation, forecast)
    print(f'Mean Absolute Percentage Error (MAPE): {round(mape,4)}')
    
    return mae, mape

In [104]:
evaluation(test, predict)

Mean Absolute Error (MAE): 1.5168
Mean Squared Error (MSE): 3.7334
Mean Absolute Percentage Error (MAPE): 1.0417


(1.5167663737970392, 1.0417192470560188)